# SVD jupyter

# Import libraries

Start by importing pytorch & matplotlib. I could not make the latex
interpreter work in binder, so the corresponding lines are commented
out.

In [ ]:
import torch 
import matplotlib.pyplot as plt
# plt.rcParams['svg.fonttype'] = 'none'
# from IPython.display import set_matplotlib_formats
# set_matplotlib_formats('svg')
# plt.rcParams.update({
#     "text.usetex": True,
#     "font.family": "Helvetica"
# })
# import matplotlib


In [ ]:
# matplotlib.rcParams["text.usetex"] = True
# matplotlib.rcParams["font.family"] = "serif"
# matplotlib.rcParams["font.size"] = "14"



# Create the functions

The separability proprieties of several functions are investigated. This
code first shows the separability proprieties of different ways of
clustering the 1D space into two regions. Two types of functions are
used to do so: \* A sharp step function (Heaviside), \* A smooth one
(Tanh).

In both cases the position of the jump is parametrised by a scalar
parameter $\alpha$.

Furhter investigations are conducted on moving front(s) with a \* A
gaussian function that is moving \* Two gaussian functions moving at
different rates

In [ ]:
L = 10                                      # Space domain
Alpha_vect = torch.linspace(0,1,1500)       # vector of alphas
x_vect = torch.linspace(0,L,2000)           # vector of x


Function = 'Heaviside'                      # Alpha-parameterised step function
Function = 'Tanh'                           # smooth alpha-parameterised step function
# Function = 'Gauss'                        # Alpha-parameterised front function
Function = 'Gauss_sum'                      # Double alpha-parameterised front functions




if Function == 'Heaviside':
    F = torch.heaviside((x_vect[:,None] - (1-Alpha_vect[None,:])*L), x_vect[-1]/x_vect[-1])
elif Function == 'Tanh':
    F = torch.tanh((x_vect[:,None] - (1-Alpha_vect[None,:])*L))
elif Function == 'Gauss':
    F = torch.exp(-(x_vect[:,None] - (1-Alpha_vect[None,:])*L)**2)

elif Function == 'Gauss_sum':
    F = torch.exp(-(x_vect[:,None] - (1-Alpha_vect[None,:])*L)**2) + torch.exp(-(x_vect[:,None] - (1-2*Alpha_vect[None,:])*L)**2) 



# Plot the reference function

In [ ]:

plt.imshow(F.t(),cmap='gray')
plt.title('Full field')
plt.xlabel('x')
plt.ylabel('\alpha')
# plt.savefig('../Results/FullField_'+Function+'.pdf', transparent=True)  
plt.show()

# Compute its SVD

In [ ]:
U, S, V = torch.svd(F)

# Compute the truncated function

From the SVD, only the $N$ first modes are kept to reconstruct the
truncated function.

In [ ]:
N = 15                                                  # Number of modes kept for the truncation                   

F_truncated = U[:,:N]@torch.diag(S[:N])@V[:,:N].t()     # Truncated function


# Plot the truncated function

In [ ]:

plt.imshow(F_truncated.t(),cmap='gray')
plt.title(f'Truncated field, N={N}')
plt.xlabel('x')
plt.ylabel('alpha')
# plt.savefig(f'../Results/TruncatedField_{N}_'+Function+'.pdf', transparent=True)  
plt.show()



# Comparison

The truncated function and its reference are compared for two values \*
$\alpha = 1/3$ & \* $\alpha = 2/3$.

In [ ]:

plt.plot(x_vect,F[:,1000],'k',label='Full, alpha = 2/3')
plt.plot(x_vect,F_truncated[:,1000],'--',label='Truncated, alpha = 2/3')
plt.plot(x_vect,F[:,500],'k',label='Full, alpha = 1/3')
plt.plot(x_vect,F_truncated[:,500],'--',label='Truncated, alpha = 1/3')
plt.legend(loc="upper left")
plt.title(f'2 slices of the field, N={N}')
plt.xlabel('x')
plt.ylabel('f(x,alpha)')
# plt.savefig(f'../Results/Sliced_TruncatedField_{N}_'+Function+'.pdf', transparent=True)  
plt.show()

# Show the decay of the singular values

The decay of the singular values gives a good insight into the
investigated function’s separability proprieties.

In [ ]:

plt.semilogy(S)
plt.ylabel('sigma_i^2')
plt.xlabel('Modes')
# plt.savefig(f'../Results/SVD_Decay_'+Function+'.pdf', transparent=True)  
plt.show()

# Interactive plot comparing the truncated function and the reference

This interactive plot allows to change the number of modes in the
truncation and the value of the parameter $\alpha$ .

In [ ]:

from ipywidgets import interact, widgets
import torch

def interactive_plot(N,alpha):
    
    # Calculate the F_trunc
    F_truncated = U[:,:N]@torch.diag(S[:N])@V[:,:N].t()

    # Plot the function
    plt.plot(x_vect,F[:,alpha])
    plt.plot(x_vect,F_truncated[:,alpha])
    plt.xlabel('x')
    plt.ylabel('f(x)')
    plt.show()

# Create an interactive slider
slider_E1 = widgets.IntSlider(value=0, min=1, max=50, step=1, description='N modes')
slider_E2 = widgets.IntSlider(value=0, min=1, max=1499, step=1, description='alpha')


# Connect the slider to the interactive plot function
interactive_plot_widget = interact(interactive_plot, N=slider_E1, alpha=slider_E2)
